In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
input_file = "20200721_2129PM/output_20200721_2129PM_1000jobs.csv"
#input_file = "output_20200723_2154PM-500jobs/output_20200723_2154PM_500jobs.csv"
#input_file = "./output_20200219_2125PM.csv"
df = pd.read_csv(input_file, header = 0)

In [3]:
original_headers = list(df.columns.values)
print(original_headers)
print(df.shape)

['root_xwf_id', 'job_id', 'start_time', 'end_time', 'submit_host', 'submit_user', 'execution_host', 'execution_user', 'job_type', 'job_exit_code', 'bytes', 'lfn', 'src_label', 'src_url', 'src_proto_host', 'dst_label', 'dst_url', 'dst_proto_host', 'transfer_success', 'checksum_success', 'actual_checksum', 'expected_checksum', 'scenario', 'corrupt_label', 'corrupt_start', 'corrupt_end', 'corrupt_rate']
(368313, 27)


In [4]:
df['flow'] = df['submit_host']+'-'+df['execution_host']
list(df.columns.values)

['root_xwf_id',
 'job_id',
 'start_time',
 'end_time',
 'submit_host',
 'submit_user',
 'execution_host',
 'execution_user',
 'job_type',
 'job_exit_code',
 'bytes',
 'lfn',
 'src_label',
 'src_url',
 'src_proto_host',
 'dst_label',
 'dst_url',
 'dst_proto_host',
 'transfer_success',
 'checksum_success',
 'actual_checksum',
 'expected_checksum',
 'scenario',
 'corrupt_label',
 'corrupt_start',
 'corrupt_end',
 'corrupt_rate',
 'flow']

In [5]:
# summarize the class distribution
#target = df.values[:,-5]
target = df['corrupt_label']
counter = Counter(target)
label_prob_dict={}
for k,v in counter.items():
    per = v / len(target) * 100
    print('Class=%s, Count=%d, Percentage=%.3f%%' % (k, v, per))
    df_c=df[(df['corrupt_label']==k)]
    target_flow=df_c['flow']
    counter_flow = Counter(target_flow)
    flow_prob_dict={}
    for i,j in counter_flow.items():
        print(i+":"+str(j)+":"+str(len(target_flow)))
        per_flow = j / len(target_flow) * 100
        print('flow=%s, Count=%d, Per__flow=%.3f%%' % (i, j, per_flow))
        df_flow=df_c[(df_c['flow']==i)]
        flow_count=df_flow['checksum_success'].value_counts(normalize=True)
        #print(flow_count.index.tolist())
        #print(flow_count.values.tolist())
        flow_count_dict=flow_count.to_dict()
        print(flow_count_dict)
        if False in flow_count_dict:
            flow_prob_dict[i]=flow_count_dict[False]
        else:
            flow_prob_dict[i]=0
    label_prob_dict[k]=flow_prob_dict

Class=nan, Count=23694, Percentage=6.433%
Class=syr-compute-c2 , Count=24, Percentage=0.007%
uc-submit-syr-compute-c2:24:24
flow=uc-submit-syr-compute-c2, Count=24, Per__flow=100.000%
{False: 1.0}
Class=ucsd-compute-c3 , Count=56, Percentage=0.015%
uc-submit-ucsd-compute-c3:56:56
flow=uc-submit-ucsd-compute-c3, Count=56, Per__flow=100.000%
{False: 1.0}
Class=unl-compute-c1 , Count=5, Percentage=0.001%
uc-submit-unl-compute-c1:5:5
flow=uc-submit-unl-compute-c1, Count=5, Per__flow=100.000%
{False: 1.0}
Class=cenic.Link9 , Count=7000, Percentage=1.901%
uc-submit-syr-compute-c2:2331:7000
flow=uc-submit-syr-compute-c2, Count=2331, Per__flow=33.300%
{True: 1.0}
uc-submit-unl-compute-c1:2331:7000
flow=uc-submit-unl-compute-c1, Count=2331, Per__flow=33.300%
{True: 1.0}
uc-submit-ucsd-compute-c3:2338:7000
flow=uc-submit-ucsd-compute-c3, Count=2338, Per__flow=33.400%
{True: 1.0}
Class=cenic.Link12 , Count=21001, Percentage=5.702%
uc-submit-syr-compute-c2:7098:21001
flow=uc-submit-syr-compute-c2,

uc-submit-ucsd-compute-c3:2331:7000
flow=uc-submit-ucsd-compute-c3, Count=2331, Per__flow=33.300%
{True: 1.0}
uc-submit-syr-compute-c2:2338:7000
flow=uc-submit-syr-compute-c2, Count=2338, Per__flow=33.400%
{True: 1.0}
uc-submit-unl-compute-c1:2331:7000
flow=uc-submit-unl-compute-c1, Count=2331, Per__flow=33.300%
{True: 1.0}
Class=ucsd-compute-c3.Link23 , Count=18872, Percentage=5.124%
uc-submit-ucsd-compute-c3:16912:18872
flow=uc-submit-ucsd-compute-c3, Count=16912, Per__flow=89.614%
{True: 0.5714285714285714, False: 0.42857142857142855}
uc-submit-unl-compute-c1:980:18872
flow=uc-submit-unl-compute-c1, Count=980, Per__flow=5.193%
{True: 1.0}
uc-submit-syr-compute-c2:980:18872
flow=uc-submit-syr-compute-c2, Count=980, Per__flow=5.193%
{True: 1.0}
Class=ucsd-router-n3.Link23 , Count=7000, Percentage=1.901%
uc-submit-unl-compute-c1:2345:7000
flow=uc-submit-unl-compute-c1, Count=2345, Per__flow=33.500%
{True: 1.0}
uc-submit-syr-compute-c2:2331:7000
flow=uc-submit-syr-compute-c2, Count=2331

In [6]:
label_prob_dict

{nan: {},
 'syr-compute-c2 ': {'uc-submit-syr-compute-c2': 1.0},
 'ucsd-compute-c3 ': {'uc-submit-ucsd-compute-c3': 1.0},
 'unl-compute-c1 ': {'uc-submit-unl-compute-c1': 1.0},
 'cenic.Link9 ': {'uc-submit-syr-compute-c2': 0,
  'uc-submit-unl-compute-c1': 0,
  'uc-submit-ucsd-compute-c3': 0},
 'cenic.Link12 ': {'uc-submit-syr-compute-c2': 0.42857142857142855,
  'uc-submit-ucsd-compute-c3': 0.4285087077418411,
  'uc-submit-unl-compute-c1': 0.42857142857142855},
 'cenic.Link0 ': {'uc-submit-ucsd-compute-c3': 0,
  'uc-submit-unl-compute-c1': 0,
  'uc-submit-syr-compute-c2': 0},
 'cenic.Link1 ': {'uc-submit-unl-compute-c1': 0,
  'uc-submit-ucsd-compute-c3': 0,
  'uc-submit-syr-compute-c2': 0},
 'unl-router-n1.Link16 ': {'uc-submit-unl-compute-c1': 0,
  'uc-submit-syr-compute-c2': 0,
  'uc-submit-ucsd-compute-c3': 0},
 'unl-router-n1.Link14 ': {'uc-submit-unl-compute-c1': 0.42857142857142855,
  'uc-submit-ucsd-compute-c3': 0,
  'uc-submit-syr-compute-c2': 0},
 'internet2.Link6 ': {'uc-submi

In [7]:
df_prob = pd.DataFrame.from_dict(label_prob_dict, orient='index')
print(type(df_prob))
df_prob

<class 'pandas.core.frame.DataFrame'>


,uc-submit-syr-compute-c2,uc-submit-ucsd-compute-c3,uc-submit-unl-compute-c1
syr-compute-c2,1.000000,NaN,NaN
cenic.Link9,0.000000,0.000000,0.000000
cenic.Link12,0.428571,0.428509,0.428571
cenic.Link0,0.000000,0.000000,0.000000
cenic.Link1,0.000000,0.000000,0.000000
unl-router-n1.Link16,0.000000,0.000000,0.000000
unl-router-n1.Link14,0.000000,0.000000,0.428571
internet2.Link6,0.000000,0.000000,0.000000
internet2.Link2,0.000000,0.000000,0.000000
internet2.Link14,0.000000,0.000000,0.000000


In [8]:
X=df_prob.to_numpy()
X=np.nan_to_num(X)
print(X)
y=df_prob.index
print(y)

[[1.         0.         0.        ]
 [0.         0.         0.        ]
 [0.42857143 0.42850871 0.42857143]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.42857143]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.57142857]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.42857143 0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.57142857 0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.42857143 0.         0.        ]
 [0.         0.         0.        ]
 [0.57142857 0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.42857143 0.        ]
 [0.         0.         0.        ]
 [0.         0.42857143 0.        ]
 [0.         0.         0.42

In [9]:
df_prob_test=df_prob[(df_prob['uc-submit-syr-compute-c2']!=0) | (df_prob['uc-submit-ucsd-compute-c3']!=0) | (df_prob['uc-submit-unl-compute-c1']!=0)]

In [10]:
X_test=df_prob_test.to_numpy()
X_test=np.nan_to_num(X_test)
print(X_test)
y_test=df_prob_test.index

[[1.         0.         0.        ]
 [0.42857143 0.42850871 0.42857143]
 [0.         0.         0.42857143]
 [0.         0.         0.57142857]
 [0.         0.42857143 0.        ]
 [0.57142857 0.         0.        ]
 [0.42857143 0.         0.        ]
 [0.57142857 0.         0.        ]
 [0.         0.42857143 0.        ]
 [0.         0.42857143 0.        ]
 [0.         0.         0.42857143]
 [0.42857143 0.71428571 0.42857143]
 [0.         1.         0.        ]
 [0.         0.         1.        ]]


In [11]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

In [12]:
kernel = 1.0 * RBF([1.0])
gpc_rbf_isotropic = GaussianProcessClassifier(kernel=kernel).fit(X_test, y_test)


In [13]:
score=gpc_rbf_isotropic.score(X,y)
print(score)

0.09090909090909091


In [14]:
gpc_rbf_isotropic.score(X_test, y_test)

0.21428571428571427

In [15]:
df_c=df[(df['corrupt_label']==k)]
print(df_c['checksum_success'].value_counts())
display(df_c)

True    7000
Name: checksum_success, dtype: int64


,root_xwf_id,job_id,start_time,end_time,submit_host,submit_user,execution_host,execution_user,job_type,job_exit_code,...,transfer_success,checksum_success,actual_checksum,expected_checksum,scenario,corrupt_label,corrupt_start,corrupt_end,corrupt_rate,flow
361313,80f059e4-4140-4ba7-b8eb-cad3520eecd7,job_sh_ID0000919,1595511358,1595511422,uc-submit,ericafu,syr-compute-c2,ericafu,compute,0,...,True,True,2416276f6d923db58f9a998346cfd386c4c33188b92461...,2416276f6d923db58f9a998346cfd386c4c33188b92461...,bypass,uc-router-n0.Link18,1.595511e+09,1.595517e+09,0.02,uc-submit-syr-compute-c2
361314,80f059e4-4140-4ba7-b8eb-cad3520eecd7,job_sh_ID0000919,1595511358,1595511422,uc-submit,ericafu,syr-compute-c2,ericafu,compute,0,...,True,True,66a1e5e51e526d106f1d9cd00a6c2cd05ba536c698b6c6...,66a1e5e51e526d106f1d9cd00a6c2cd05ba536c698b6c6...,bypass,uc-router-n0.Link18,1.595511e+09,1.595517e+09,0.02,uc-submit-syr-compute-c2
361315,80f059e4-4140-4ba7-b8eb-cad3520eecd7,job_sh_ID0000919,1595511358,1595511422,uc-submit,ericafu,syr-compute-c2,ericafu,compute,0,...,True,True,0bab203b629d88b4ce760122fddeecc39879f31389e1a7...,0bab203b629d88b4ce760122fddeecc39879f31389e1a7...,bypass,uc-router-n0.Link18,1.595511e+09,1.595517e+09,0.02,uc-submit-syr-compute-c2
361316,80f059e4-4140-4ba7-b8eb-cad3520eecd7,job_sh_ID0000919,1595511358,1595511422,uc-submit,ericafu,syr-compute-c2,ericafu,compute,0,...,True,True,bf239774fe1bcf1895e2c3c550c1f4f0be5edc223ebaee...,bf239774fe1bcf1895e2c3c550c1f4f0be5edc223ebaee...,bypass,uc-router-n0.Link18,1.595511e+09,1.595517e+09,0.02,uc-submit-syr-compute-c2
361317,80f059e4-4140-4ba7-b8eb-cad3520eecd7,job_sh_ID0000919,1595511358,1595511422,uc-submit,ericafu,syr-compute-c2,ericafu,compute,0,...,True,True,096a0681b644e28d0082f26ac153459ed15484ba26201f...,096a0681b644e28d0082f26ac153459ed15484ba26201f...,bypass,uc-router-n0.Link18,1.595511e+09,1.595517e+09,0.02,uc-submit-syr-compute-c2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368308,80f059e4-4140-4ba7-b8eb-cad3520eecd7,job_sh_ID0000181,1595516805,1595516868,uc-submit,ericafu,unl-compute-c1,ericafu,compute,0,...,True,True,0bab203b629d88b4ce760122fddeecc39879f31389e1a7...,0bab203b629d88b4ce760122fddeecc39879f31389e1a7...,bypass,uc-router-n0.Link18,1.595511e+09,1.595517e+09,0.02,uc-submit-unl-compute-c1
368309,80f059e4-4140-4ba7-b8eb-cad3520eecd7,job_sh_ID0000181,1595516805,1595516868,uc-submit,ericafu,unl-compute-c1,ericafu,compute,0,...,True,True,bf239774fe1bcf1895e2c3c550c1f4f0be5edc223ebaee...,bf239774fe1bcf1895e2c3c550c1f4f0be5edc223ebaee...,bypass,uc-router-n0.Link18,1.595511e+09,1.595517e+09,0.02,uc-submit-unl-compute-c1
368310,80f059e4-4140-4ba7-b8eb-cad3520eecd7,job_sh_ID0000181,1595516805,1595516868,uc-submit,ericafu,unl-compute-c1,ericafu,compute,0,...,True,True,096a0681b644e28d0082f26ac153459ed15484ba26201f...,096a0681b644e28d0082f26ac153459ed15484ba26201f...,bypass,uc-router-n0.Link18,1.595511e+09,1.595517e+09,0.02,uc-submit-unl-compute-c1
368311,80f059e4-4140-4ba7-b8eb-cad3520eecd7,job_sh_ID0000181,1595516805,1595516868,uc-submit,ericafu,unl-compute-c1,ericafu,compute,0,...,True,True,7902fe2217fc7ac847e40d8b7fdec6bf820520918be642...,7902fe2217fc7ac847e40d8b7fdec6bf820520918be642...,bypass,uc-router-n0.Link18,1.595511e+09,1.595517e+09,0.02,uc-submit-unl-compute-c1


In [16]:
df_c=df[(df['checksum_success']==False)&(df['execution_host']=='syr-compute-c2')&(df['corrupt_label']=='cenic.Link12 ')]
print(len(df))
print(len(df_c))
#df_c=df[(df['corrupt_label']=="unl-compute-c1")]
display(df_c)

368313
3042


,root_xwf_id,job_id,start_time,end_time,submit_host,submit_user,execution_host,execution_user,job_type,job_exit_code,...,transfer_success,checksum_success,actual_checksum,expected_checksum,scenario,corrupt_label,corrupt_start,corrupt_end,corrupt_rate,flow
30779,a0a8aef5-6b4b-4d87-922a-644331b06fcf,job_sh_ID0000917,1595389764,1595389768,uc-submit,ericafu,syr-compute-c2,ericafu,compute,256,...,True,False,f540d7e135a8326d173ca844a7d12c99f65df3325556a3...,2416276f6d923db58f9a998346cfd386c4c33188b92461...,bypass,cenic.Link12,1.595390e+09,1.595391e+09,0.02,uc-submit-syr-compute-c2
30782,a0a8aef5-6b4b-4d87-922a-644331b06fcf,job_sh_ID0000917,1595389764,1595389768,uc-submit,ericafu,syr-compute-c2,ericafu,compute,256,...,True,False,fb8550e0e986605b6cca462b41feaddaedeb4d35be79a5...,bf239774fe1bcf1895e2c3c550c1f4f0be5edc223ebaee...,bypass,cenic.Link12,1.595390e+09,1.595391e+09,0.02,uc-submit-syr-compute-c2
30783,a0a8aef5-6b4b-4d87-922a-644331b06fcf,job_sh_ID0000917,1595389764,1595389768,uc-submit,ericafu,syr-compute-c2,ericafu,compute,256,...,True,False,42913c6698b4595e7c6c3652dd133b5b52cf165484ef77...,096a0681b644e28d0082f26ac153459ed15484ba26201f...,bypass,cenic.Link12,1.595390e+09,1.595391e+09,0.02,uc-submit-syr-compute-c2
30800,a0a8aef5-6b4b-4d87-922a-644331b06fcf,job_sh_ID0000919,1595389764,1595389768,uc-submit,ericafu,syr-compute-c2,ericafu,compute,256,...,True,False,f540d7e135a8326d173ca844a7d12c99f65df3325556a3...,2416276f6d923db58f9a998346cfd386c4c33188b92461...,bypass,cenic.Link12,1.595390e+09,1.595391e+09,0.02,uc-submit-syr-compute-c2
30803,a0a8aef5-6b4b-4d87-922a-644331b06fcf,job_sh_ID0000919,1595389764,1595389768,uc-submit,ericafu,syr-compute-c2,ericafu,compute,256,...,True,False,fb8550e0e986605b6cca462b41feaddaedeb4d35be79a5...,bf239774fe1bcf1895e2c3c550c1f4f0be5edc223ebaee...,bypass,cenic.Link12,1.595390e+09,1.595391e+09,0.02,uc-submit-syr-compute-c2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51748,a0a8aef5-6b4b-4d87-922a-644331b06fcf,job_sh_ID0000189,1595390736,1595390740,uc-submit,ericafu,syr-compute-c2,ericafu,compute,256,...,True,False,fb8550e0e986605b6cca462b41feaddaedeb4d35be79a5...,bf239774fe1bcf1895e2c3c550c1f4f0be5edc223ebaee...,bypass,cenic.Link12,1.595390e+09,1.595391e+09,0.02,uc-submit-syr-compute-c2
51749,a0a8aef5-6b4b-4d87-922a-644331b06fcf,job_sh_ID0000189,1595390736,1595390740,uc-submit,ericafu,syr-compute-c2,ericafu,compute,256,...,True,False,42913c6698b4595e7c6c3652dd133b5b52cf165484ef77...,096a0681b644e28d0082f26ac153459ed15484ba26201f...,bypass,cenic.Link12,1.595390e+09,1.595391e+09,0.02,uc-submit-syr-compute-c2
51752,a0a8aef5-6b4b-4d87-922a-644331b06fcf,job_sh_ID0000181,1595390736,1595390739,uc-submit,ericafu,syr-compute-c2,ericafu,compute,256,...,True,False,f540d7e135a8326d173ca844a7d12c99f65df3325556a3...,2416276f6d923db58f9a998346cfd386c4c33188b92461...,bypass,cenic.Link12,1.595390e+09,1.595391e+09,0.02,uc-submit-syr-compute-c2
51755,a0a8aef5-6b4b-4d87-922a-644331b06fcf,job_sh_ID0000181,1595390736,1595390739,uc-submit,ericafu,syr-compute-c2,ericafu,compute,256,...,True,False,fb8550e0e986605b6cca462b41feaddaedeb4d35be79a5...,bf239774fe1bcf1895e2c3c550c1f4f0be5edc223ebaee...,bypass,cenic.Link12,1.595390e+09,1.595391e+09,0.02,uc-submit-syr-compute-c2


In [17]:
df_c=df[(df['corrupt_label']=="internet2.Link2")]
display(df_c)

,root_xwf_id,job_id,start_time,end_time,submit_host,submit_user,execution_host,execution_user,job_type,job_exit_code,...,transfer_success,checksum_success,actual_checksum,expected_checksum,scenario,corrupt_label,corrupt_start,corrupt_end,corrupt_rate,flow


In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree
import sklearn
import graphviz 
from sklearn.calibration import CalibratedClassifierCV

In [19]:
clf_prob = DecisionTreeClassifier(max_depth = 20)
clf_model=clf_prob.fit(X, y)

In [20]:
score=clf_prob.score(X_test, y_test)
print(score)

0.7142857142857143


In [21]:
df_failure = df[ (df.checksum_success==False)]
print(df.shape)
print(df_failure.shape)
df_dummy = pd.get_dummies(df_failure[['submit_host', 'execution_host','src_label','dst_label','transfer_success','checksum_success','corrupt_label']], prefix=['SRC', 'DEST','SRC_L','DEST_L'], columns=['submit_host', 'execution_host','src_label','dst_label'])

(368313, 28)
(92269, 28)


In [22]:
display(df_dummy)

,transfer_success,checksum_success,corrupt_label,SRC_uc-submit,DEST_syr-compute-c2,DEST_ucsd-compute-c3,DEST_unl-compute-c1,SRC_L_condorpool,DEST_L_condorpool
366,True,False,syr-compute-c2,1,1,0,0,1,1
367,True,False,syr-compute-c2,1,1,0,0,1,1
369,True,False,syr-compute-c2,1,1,0,0,1,1
387,True,False,syr-compute-c2,1,1,0,0,1,1
388,True,False,syr-compute-c2,1,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...
354303,True,False,uc-router-n0.Link19,1,1,0,0,1,1
354304,True,False,uc-router-n0.Link19,1,1,0,0,1,1
354307,True,False,uc-router-n0.Link19,1,1,0,0,1,1
354310,True,False,uc-router-n0.Link19,1,1,0,0,1,1


In [23]:
y=df_failure['corrupt_label']
x=df_dummy.drop(['corrupt_label'],axis=1)

In [24]:
%%time
clf = DecisionTreeClassifier(max_depth = 20)
clf_model=clf.fit(x,y)

CPU times: user 153 ms, sys: 6.56 ms, total: 160 ms
Wall time: 159 ms


In [25]:
def accuracy(model, test_data, classes, k):
    if(k==0): return null,null
    num = classes.size
    correct_class={}
    correct=0
    for label in classes:
        label_pred=[]
        isCorrect=False
        test_data_1=test_data[test_data.corrupt_label==label]
        if test_data_1.shape[0]==0:
            print("No match")
            continue
        x_test_1=test_data_1.drop(['corrupt_label'],axis=1)
        test_prob_1 = model.predict_proba(x_test_1)
        print("shape:"+str(test_prob_1.shape))
        print(test_prob_1)
        label_array=test_prob_1.mean(axis=0)
        #label_array=np.nanmean(np.where(test_rf_prob_1!=0,test_rf_prob_1,np.nan),0)
        label_index = label_array.argmax()
        label_index_sort = label_array.argsort()
        #label_pred[0] = classes[label_index]
        for j in range(0,k): 
            if(j==0):
                label_pred.append(classes[label_index])
            else:
                label_pred.append(classes[label_index_sort[(-1)*j-1]])
            #print(j,label_pred[j],label)
            if(label==label_pred[j]):
                isCorrect=True
                break
               
        #print(label_pred)
        #print(isCorrect)
        if isCorrect:
            correct=correct+1
            correct_class[label]=1
        else:
            correct_class[label]=0
    return correct_class, correct, correct/num 

In [27]:
c,correct,accu=accuracy(clf_model, df_dummy, clf.classes_,2)
print(str(correct)+":"+str(accu))
c

shape:(9000, 14)
[[9.35309310e-02 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [9.35309310e-02 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [9.35309310e-02 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [1.01033678e-01 0.00000000e+00 3.20106702e-01 ... 1.66722241e-04
  2.39879960e-01 2.39879960e-01]
 [1.01033678e-01 0.00000000e+00 3.20106702e-01 ... 1.66722241e-04
  2.39879960e-01 2.39879960e-01]
 [1.01033678e-01 0.00000000e+00 3.20106702e-01 ... 1.66722241e-04
  2.39879960e-01 2.39879960e-01]]
shape:(7215, 14)
[[0.09840363 0.24248026 0.         ... 0.         0.         0.        ]
 [0.09840363 0.24248026 0.         ... 0.         0.         0.        ]
 [0.09840363 0.24248026 0.         ... 0.         0.         0.        ]
 ...
 [0.09840363 0.24248026 0.         ... 0.         0.         0.        ]
 [0.09840363 0.24248026 0.         ... 0.         0.         0.      

{'cenic.Link12 ': 0,
 'esnet.Link1 ': 0,
 'internet2.Link0 ': 1,
 'starlight.Link9 ': 1,
 'syr-compute-c2 ': 0,
 'syr-compute-c2.Link26 ': 0,
 'syr-router-n2.Link24 ': 0,
 'uc-router-n0.Link19 ': 1,
 'ucsd-compute-c3 ': 0,
 'ucsd-compute-c3.Link23 ': 1,
 'ucsd-router-n3.Link22 ': 0,
 'unl-compute-c1 ': 0,
 'unl-compute-c1.Link16 ': 0,
 'unl-router-n1.Link14 ': 1}